# Implementation of Convolutional Neural Networks

In [1]:
import numpy as np
np.random.seed(1)

In [2]:
def conv_one_step(image_slice, W, b):
    s = W * image_slice      #image_slice.shape = (f,f)
    Z = s.sum()
    Z = np.squeeze(Z+b)
    return Z

In [3]:
def conv2D(images, W, b, hparameters):
    stride = hparameters['stride']
    pad = hparameters['pad']
    m = images.shape[0]         # m denotes no of images
    H,Wid,C = W.shape
    
    # Padding height and width
    images_pad = np.pad(images, ((0,0),(pad,pad),(pad,pad)), 'constant', constant_values=(0))
    
    # Initiating zero matrix of size of Convolutional Block
    Z = np.zeros((m,16,16,4))
    
    for i in range(m): 
        image_pad = images_pad[i]
        for h in range(H):
            for w in range(Wid):
                for c in range(C):
                    vert_start = h * stride
                    vert_end = vert_start + 3
                    hori_start = w * stride
                    hori_end = hori_start + 3
                    image_slice = image_pad[vert_start:vert_end, hori_start:hori_end]
                    Z[i,h, w, c] = conv_one_step(image_slice,W[:,:,c],b[:,:,c])
    cache = (images, W, b, hparameters)        # Will be used during backprop
    return Z, cache

## Let's initialize random weights for computation

In [4]:
# For conv block
W = np.random.randn(3,3,4)
b = np.random.randn(1,1,4)

# For first fully connected layer
W_fc_1024 = np.random.randn(10,1024)
b_fc_1024 = np.zeros((10,1))

# For second fully connected layer
W_10 = np.random.randn(1,10)
b_10 = np.zeros((1,1))

### Let's check our convolutional block and find A using relu activation

In [5]:
np.random.seed(1)
no_of_images = m = 2
images = np.random.randn(m,32,32)  # m images of (32,32)
hparameters = {"pad" : 1,
               "stride": 2}

Z, cache_conv = conv2D(images, W, b, hparameters)
A = np.maximum(0,Z)
assert(A.shape == Z.shape)
activation_cache = Z        # Will be used for backpropogation
print(A.shape)

(2, 16, 16, 4)



Shape of A comes out to be (2,16,16,4) which is same as what we expected.

## Fully connected layers
* For first fully connected layer we just need to reshape it.
* For second fc layer, we compute W*X + b

In [6]:
fc_1024 = np.reshape(A, (m,-1))
assert(fc_1024.shape[1] == 1024)
fc_cache1 = fc_1024     # Will be used while backpropogation

fc_10 = W_fc_1024.dot(fc_1024.T) + b_fc_1024
fc_cache2 = fc_10       # Will be used while backpropogation

y = W_10.dot(fc_10) + b_10
print(y)

[[-12.89682204  13.37765949]]


### Calculating loss
Now, we have our predicted output. Let's define MSE loss function and calculate loss comapring to some random Y_test.

In [7]:
 def MSE_loss(Y, Y_pred):
     return np.square(Y - Y_pred).mean()

In [8]:
Y_pred = np.random.rand(m) * 10
print(MSE_loss(y, Y_pred))

220.13788448


## Success
**Now, we have successfully implemented forward propogation of Convolutional Neural Nets.**